In [5]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.vector_stores import ChromaVectorStore
from llama_index.readers.chroma import ChromaReader
from llama_index import StorageContext, load_index_from_storage, load_indices_from_storage
# from transformers import AutoTokenizer, AutoModel
from llama_index.embeddings import HuggingFaceEmbedding
from llama_index.node_parser import SentenceSplitter 
from llama_index.schema import MetadataMode
from IPython.display import Markdown, display
import chromadb
import pandas as pd 
import openai
import os
import getpass

In [2]:
data_path = os.path.join('/workspace/data/')
index_path = os.path.join('/workspace/db/local')

In [3]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")
openai.api_key = os.environ["OPENAI_API_KEY"]

OpenAI API Key: ········


In [4]:
model_name = 'kakaobank/kf-deberta-base'
embed_model = HuggingFaceEmbedding(model_name=model_name)

In [6]:
parser = SentenceSplitter(chunk_size=512, chunk_overlap=30)   # SentenceSplitter(chunk_size=1024, chunk_overlap=20)

In [7]:
desc_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'desc'))
features_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'features'))
qualification_storage_context = StorageContext.from_defaults(persist_dir=os.path.join(index_path, 'qualification'))

In [8]:
service_context = ServiceContext.from_defaults(node_parser=parser, embed_model=embed_model)

In [9]:
# service_context 전달 안해주면 query 시 dimension 오류 발생 
features_idx = load_index_from_storage(features_storage_context, index_id='loan_tmp', service_context=service_context)
desc_idx = load_indices_from_storage(desc_storage_context, index_ids=['card_tmp', 'loan_tmp', 'deposit_tmp'], service_context=service_context)
qualification_idx = load_index_from_storage(qualification_storage_context, index_id='loan_tmp', service_context=service_context)

In [10]:
query_engine = desc_idx[0].as_query_engine()
query_engine.query('보증 자격')

Response(response='서민금융진흥원의 보증을 통해 신청 가능한 카드의 보증 자격은 군인연금수급자임을 증명하는 신분증을 가지고 있는 사람들입니다. 이러한 사람들은 전국 군복지시설 및 5대 대표업종 청구할인, 버스요금 지원서비스를 받을 수 있습니다.', source_nodes=[NodeWithScore(node=TextNode(id_='902300ff-cbca-47f7-bd8d-5235a4c79b5c', embedding=None, metadata={'category': 'card'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='카드_1016', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'card'}, hash='f1d7cec412088a7b70fb8e6387a4e2eb1e735924b3add09139ec520f6b849c1f'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='5cd004e9-c047-437d-a69c-60ae0e78bff8', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'card'}, hash='07f50829d05259589f87b15fbb2aa65512a9f817d23e35896e89eed6b02d4a86'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='206c2c2c-0d11-47f9-a8e9-af8b852845ee', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='4c30177fed83222468f96faa993e4a64ea700

### Retrieve 

In [13]:
from llama_index.retrievers import VectorIndexRetriever 
from llama_index.query_engine import RetrieverQueryEngine 
from llama_index.postprocessor import SimilarityPostprocessor 

In [19]:
retriever = VectorIndexRetriever(
    index = features_idx,
    service_context=service_context,
    similarity_top_k = 10, 
    verbose=True
)

In [20]:
from llama_index import QueryBundle 

nodes = retriever.retrieve(QueryBundle('보증 대출'))

In [29]:
print(f'num of nodes: {len(nodes)}')
nodes[1].text, nodes[1].score

num of nodes: 10


('국민행복기금으로부터 신용보증서를 발급 받은 고객의 연20%이상의 고금리대출을 은행의 저금리 대출로 전환해주는 대출 상품',
 0.721333835387794)

### PostProcessor 

In [28]:
from llama_index.postprocessor import SimilarityPostprocessor, CohereRerank
from llama_index.schema import Node, NodeWithScore 

In [33]:
# similarity post processor 
processor = SimilarityPostprocessor(similarity_cutoff=0.71)
filtered = processor.postprocess_nodes(nodes)

In [45]:
filtered[0]

NodeWithScore(node=TextNode(id_='5a8b8c76-d97f-4ee7-87ee-aeb3e60cdacd', embedding=None, metadata={'category': 'loan'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=['category'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='대출_852', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'category': 'loan'}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='95435469-e1e8-428f-ade6-2ee2101f5924', node_type=<ObjectType.TEXT: '1'>, metadata={'category': 'loan'}, hash='efbbf86cada436a84c1203c58317827e0ca887812e09d5beb05c33accc12d6f0'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='5ffb1ab1-6099-41f3-8b92-c0eb66f1ad78', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='556a6407d88aebf899ab1c287cc6db398c447d5032a4d4a23f905fc75c6efb68')}, hash='a60af23bbf14937554f340c059c7eb99ad0f0cfcf9d3d4f6527746d2375aa6ba', text='제2금융권 전세자금대출을 전액 상환하기 위해 주택금융신용보증서를 담보로 한 대출', star

In [34]:
len(filtered)

5

In [40]:
filtered[0].score, filtered[0].text

(0.735662646015599, '제2금융권 전세자금대출을 전액 상환하기 위해 주택금융신용보증서를 담보로 한 대출')

In [56]:
from llama_index.postprocessor import KeywordNodePostprocessor 

postprocessor = KeywordNodePostprocessor(
    required_keywords=['대출']
)

len(postprocessor.postprocess_nodes(filtered))

2

### Re-ranking 